<a href="https://colab.research.google.com/github/Martinccv/Clases-DS/blob/main/Clase45_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformers para Martin Fierro

Los dos principales objetivos de este tutorial son mostrar cómo es una arquitectura transformer y cómo usar las herramientas provistas por [Hugging Face](https://huggingface.co/). En este tutorial usaremos una implementación de [GPT2](https://openai.com/blog/better-language-models/) en español para generar (una vez más) texto similar al Martín Fierro, que se puede descargar [aquí](https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/martin_fierro.txt).

Créditos:
*   [Johanna Frau y Mauricio Mazuecos - DiploDatos 2022](https://github.com/DiploDatos/AprendizajeProfundo)
*   [Jay Alammar](https://jalammar.github.io/illustrated-gpt2/) por las imágenes.

In [1]:
!pip install transformers

In [2]:
import torch
import numpy as np
import pandas as pd
import transformers
import torch.nn as nn
import os
import sys
from time import time
import unicodedata, re

use_cuda = torch.cuda.is_available()

## GPT2

GPT2 es un modelo de generación de texto basado en transformers. A diferencia de BERT u otros transformers, GPT2 está basado en sucesivas instancias de decoders

<img src="images/gpt-2-transformer-xl-bert-3.png"
     alt="GPT2 Decoder Architecture"
     style="float: center; margin-right: 150px;"
     width=75%/>
     
<img src="images/gpt2_function.gif"
     alt="GPT2 Text Generation Function"
     style="float: center; margin-right: 150px;"
     width=75%/>
     
**Recursos:**

[The Illustrated GPT-2 (Visualizing Transformer Language Models)](https://jalammar.github.io/illustrated-gpt2/)

## Parte 1: Usando transformers de Hugging Face

La librería transformers permite usar modelos ya entrenados en un pipeline y nos facilita funciones que procesan el input, lo tokenizan y hace el forward pass para generar las predicciones.

Dada una tarea, pipeline descarga un modelo y tokenizador apropiados para la tarea. En este caso, especificamos un modelo preentrenado de GPT2-small en idioma español.

<img src="images/gpt2_sizes.png"
     alt="GPT2 Model Sizes"
     style="float: center; margin-right: 200px;"
     width=70%/>
     
<img src="images/gpt2_decoder_stacks.png"
     alt="GPT2 Model Sizes"
     style="float: center; margin-right: 200px;"
     width=70%/>
     

Para nuestra tarea en particular, pipeline tomará un texto e internamente se encargará del tokenizado de la oración y de generar texto hasta el *max_length* indicado.

In [3]:
from transformers import pipeline

generator = pipeline('text-generation', model="datificate/gpt2-small-spanish")
#generator = pipeline('text-generation', model="DeepESP/gpt2-spanish-medium")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/817 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/850k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/508k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [4]:
input_sentence = 'Aquí me pongo a'
output = generator(input_sentence, max_length=50)
print('Generated Text: %s...' % input_sentence)
print('... ', output[0]['generated_text'][len(input_sentence):])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Generated Text: Aquí me pongo a...
...   volar".

La historia continúa en varias ocasiones pero, en el siglo II d. C., los "efectos" "spongo" no se podían llevar a cabo. En el siglo IV d. C.,


Para usar otras tareas de 🤗 Transformers, [aquí](https://huggingface.co/transformers/task_summary.html) hay una lista completa de todas las tareas que se implementan.

---

## Parte 1: Importando de 🤗 Transformers

Lo primero es importar el modelo con el cuál trabajaremos. En nuestro caso usaremos [GPT2-Small en español de Datificate](https://huggingface.co/datificate/gpt2-small-spanish) (pueden jugar con el modelo desde la misma página de 🤗).

Nuestrea tarea de interés es hacer Language Modeling del Martín Fierro, por lo cual importamos **GPT2LMHeadModel**, que es una implementación de GPT2 con la última capa lineal para retornar los logits del tamaño del vocabulario.

Como el modelo *datificate/gpt2-small-spanish* fué entrenado en tensorflow, usamos el flag *from_tf=True* para asegurarnos que importe los pesos correctamente.

In [5]:
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('datificate/gpt2-small-spanish', from_tf=True)
#model = GPT2LMHeadModel.from_pretrained('DeepESP/gpt2-spanish-medium', from_tf=True)

tf_model.h5:   0%|          | 0.00/498M [00:00<?, ?B/s]

All TF 2.0 model weights were used when initializing GPT2LMHeadModel.

Some weights of GPT2LMHeadModel were not initialized from the TF 2.0 model and are newly initialized: ['lm_head.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

El paso siguiente es descargar un tokenizador. El tokenizador hará por nosotres el trabajo de separar el texto en palabras y convertirlas en sus correspondientes ids dentro del vocabulario. Al igual que con el modelo, 🤗 provee tokenizers ya implementados para sus correspondientes modelos.

In [7]:
from transformers import GPT2TokenizerFast

tokenizer = GPT2TokenizerFast.from_pretrained('datificate/gpt2-small-spanish', add_prefix_space=True)
#tokenizer = GPT2TokenizerFast.from_pretrained('DeepESP/gpt2-spanish-medium', add_prefix_space=True)


tokenized_seq = tokenizer("Yo nunca habia escuchado hablar de Innsmouth hasta")
print('inputs_ids: {}'.format(tokenized_seq['input_ids']))
print('attention_mask: {}'.format(tokenized_seq['attention_mask']))
tokens = tokenizer.convert_ids_to_tokens(tokenized_seq['input_ids'])
print('tokens: {}'.format(tokens))

inputs_ids: [4701, 2686, 498, 363, 27422, 6242, 258, 599, 10007, 15308, 657]
attention_mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
tokens: ['ĠYo', 'Ġnunca', 'Ġhab', 'ia', 'Ġescuchado', 'Ġhablar', 'Ġde', 'ĠIn', 'ns', 'mouth', 'Ġhasta']


El tokenizer se puede encargar por nosotres de manejar temas de padding y truncado entre otros. Para más info en las funciones del tokenizers, mirar [aquí](https://huggingface.co/transformers/main_classes/tokenizer.html).

El tokenizer también puede procesar múltiples oraciones al mismo tiempo y retornar tensores para el framework que estemos usando de base, sea este TensorFlow 2.0 ('*tf*') o PyTorch ('*pt*').

In [8]:
sentences = ["Yo nunca habia escuchado hablar de Innsmouth hasta",
             "Hacia el oeste de Arkham las montañas se levantaban indómitas y en sus entrañas"]

In [9]:
tokenized_seq = tokenizer(sentences,
                          padding=True,
                          return_tensors="pt")

print('inputs_ids:\n{}'.format(tokenized_seq['input_ids']))
print('attention_mask:\n{}'.format(tokenized_seq['attention_mask']))

inputs_ids:
tensor([[ 4701,  2686,   498,   363, 27422,  6242,   258,   599, 10007, 15308,
           657,     0,     0,     0,     0,     0,     0,     0,     0],
        [11945,   284,  2866,   258, 18307,  2747,   347,  6674,   306,  3620,
          4413,  1582,  1178,  2982,   287,   278,   452,  7259,  1846]])
attention_mask:
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


Incluso el tokenizer puede manejar listas de palabras con solo agregar el parámetro *is_split_into_words=True*.

In [10]:
tokenized_seq = tokenizer([sent.split() for sent in sentences],
                          padding=True,
                          return_tensors="pt",
                          is_split_into_words=True)

print('inputs_ids:\n{}'.format(tokenized_seq['input_ids']))
print('attention_mask:\n{}'.format(tokenized_seq['attention_mask']))

inputs_ids:
tensor([[ 4701,  2686,   498,   363, 27422,  6242,   258,   599, 10007, 15308,
           657,     0,     0,     0,     0,     0,     0,     0,     0],
        [11945,   284,  2866,   258, 18307,  2747,   347,  6674,   306,  3620,
          4413,  1582,  1178,  2982,   287,   278,   452,  7259,  1846]])
attention_mask:
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


---
## Parte 2: Finetune con PyTorch nativo

Para entrenar con PyTorch, necesitamos crear una instancia de un Dataset de PyTorch. Esta vez, podemos usar el tokenizer que ya importamos para hacer el encoding de las palabras.

Por simplicidad, dividiremos el el conjunto de datos en palabras y usaremos listas de palabras de tamaño fijo.

Para entrenamiento nativo en Tensorflow 2, ver [aquí](https://huggingface.co/transformers/training.html#fine-tuning-in-native-tensorflow-2).

In [11]:
from torch.utils.data import Dataset

class MartinFierroDatasetGPT2(Dataset):
    def __init__(self, textdata, maxlen, tokenizer):

        self.maxlen = maxlen
        # Get ourselves a list of words so we can iterate
        split_text = textdata.split()
        # cut the text in sequences of maxlen characters
        self.sentences = {}
        for idx, i in enumerate(range(0, len(split_text) - maxlen - 1, maxlen)):
            self.sentences[idx] = split_text[i: i + maxlen]

        # You need to activate padding in order to return tensors
        self.data = tokenizer(list(self.sentences.values()),
                              padding=True,
                              is_split_into_words=True,
                              return_tensors="pt")

        self.length = len(self.sentences)
        print('NB sequences:', self.length)

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.data.items()}
        return item


Cargamos el archivo e instanciamos el dataset

In [13]:
with open('./martin_fierro.txt', 'r') as finput:
    text = unicodedata.normalize('NFC', finput.read()).lower()
    text = re.sub('\s+', ' ', text).strip()

print('Corpus length: %d' % len(text))

train_dataset = MartinFierroDatasetGPT2(text, 50, tokenizer)

Corpus length: 61709
NB sequences: 236


Ponemos nuestro modelo en modo train y seteamos el dispositivo donde vamos a ejecutar.

In [14]:
model.train()
device = torch.device('cuda') if use_cuda else torch.device('cpu')
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

Importamos de transformers la implementacion de [AdamW](https://arxiv.org/abs/1711.05101).

In [15]:
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay=0.01)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Instanciamos nuestro dataloader con el dataset del Martín Fierro

In [16]:
from torch.utils.data import DataLoader

dataloader_config = {'dataset': train_dataset,
                     'batch_size': 8,
                     'shuffle': True,
                     'num_workers': 0,
                     'pin_memory': use_cuda}

dataloader = DataLoader(**dataloader_config)

Finalmente hacemos un forward pass por todo el dataset (i.e. entrenamos por una epoch). El modelo importado de 🤗 transformers ya se encarga de calcular la loss para la tarea que necesitamos (en este caso es un LM) en el forward pass si le pasamos el argumento *labels*.

In [17]:
from tqdm.notebook import tqdm

stream = tqdm(enumerate(dataloader), total=len(dataloader))
for i, sample in stream:
    input_ids = sample['input_ids'].to(device)
    attention_mask = sample['attention_mask'].to(device)

    outputs = model(input_ids=input_ids,
                    attention_mask=attention_mask,
                    labels=input_ids)
    optimizer.zero_grad()
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    stream.set_postfix({'loss': loss.detach().cpu().numpy()})

  0%|          | 0/30 [00:00<?, ?it/s]

<ipython-input-11-faed882ce5da>:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.data.items()}


In [18]:
num_epochs = 5
from tqdm.notebook import tqdm
from torch.nn.functional import cross_entropy

for epoch in range(num_epochs):
    dataloader = DataLoader(**dataloader_config)
    stream = tqdm(enumerate(dataloader), total=len(dataloader))
    total_perplexity = 0.0
    total_tokens = 0

    for i, sample in stream:
        input_ids = sample['input_ids'].to(device)
        attention_mask = sample['attention_mask'].to(device)

        outputs = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        labels=input_ids)  # Usamos los input_ids como etiquetas para calcular la pérdida

        loss = outputs.loss
        perplexity = cross_entropy(outputs.logits.view(-1, outputs.logits.size(-1)), input_ids.view(-1), reduction='none')
        perplexity = perplexity.mean().exp()  # Calculamos la perplejidad a partir de la loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_perplexity += perplexity.item() * input_ids.size(0)
        total_tokens += input_ids.size(0) * input_ids.size(1)  # Cantidad total de tokens en el lote

        stream.set_postfix({'loss': loss.detach().cpu().numpy(), 'epoch': epoch+1})

    avg_perplexity = total_perplexity / total_tokens
    print(f'Epoch [{epoch+1}/{num_epochs}], Average Perplexity: {avg_perplexity:.2f}')
#torch.save(model.state_dict(), "./modelo-MF-v01.pth")

  0%|          | 0/30 [00:00<?, ?it/s]

<ipython-input-11-faed882ce5da>:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.data.items()}


Epoch [1/5], Average Perplexity: 68.27


  0%|          | 0/30 [00:00<?, ?it/s]

Epoch [2/5], Average Perplexity: 57.64


...

Epoch [3/5], Average Perplexity: 51.71


  0%|          | 0/30 [00:00<?, ?it/s]

Epoch [4/5], Average Perplexity: 45.91


  0%|          | 0/30 [00:00<?, ?it/s]

Epoch [5/5], Average Perplexity: 40.76


## Parte 3: Entrenando con Transformers Trainers

🤗 también provee sus propios métodos para entrenar. En este caso:
  - TrainingArgument: nos genera una configuración para un entrenamiento
  - Trainer: clase que se encargará del entrenamiento por nosotres
  
Importamos el Data Collator necesario para nuestra tarea y pasamos los argumentos necesario para cada objeto.

In [21]:
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling

# We're not training with MLM, so we must set mlm=False
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,
                                                mlm=False)

training_args = TrainingArguments(
    output_dir='./gpt2-fierro',      # output directory
    num_train_epochs=1,              # total # of training epochs
    per_device_train_batch_size=8,   # batch size per device during training
    per_device_eval_batch_size=1,    # batch size for evaluation
    warmup_steps=100,                  # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainer = Trainer(
    model=model,                     # the instantiated 🤗 Transformers model to be trained
    args=training_args,              # training arguments, defined above
    train_dataset=train_dataset,     # training dataset (THE SAME AS BEFORE)
    data_collator=data_collator
)

Finalmente, entrenar es tan simple como llamar al método train() del trainer

In [22]:
trainer.train()

<ipython-input-11-faed882ce5da>:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.data.items()}


Step,Training Loss


TrainOutput(global_step=30, training_loss=4.969405110677084, metrics={'train_runtime': 16.4989, 'train_samples_per_second': 14.304, 'train_steps_per_second': 1.818, 'total_flos': 9876022272000.0, 'train_loss': 4.969405110677084, 'epoch': 1.0})

---
## Parte 4: Usando nuestro nuevo modelo en un pipeline

In [37]:
model.cpu()
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [38]:
input_sentence = 'Aquí me pongo a'
output = generator(input_sentence, max_length=200)
print('Generated Text: %s...' % input_sentence)
print('... ', output[0]['generated_text'][len(input_sentence):])

Generated Text: Aquí me pongo a...
...   mi vida. 224 por ser heado como un gaucho el piaro, y a pesar de los tichas, yo soy una mulaza mía y soy un hombre de paja, que lo hacía bien, pero me quería la suerte; me vino en la cruz y se tuvo un chacarcito, y las de la cueva salieron de allá. 235 me fue al campo del sol. 236 me me quedó sin saber si lo hacía, ni decir si me quería, pues no sé de dónde lo le habían de ir y donde le habían perdían a la suerte. 237 si al cabo me andaban por la suerte y si me me alquila que tenían, ya ya no lo tenía en la tierra, que no tenían la suerte, ya qué había el hombre en la cabeza cuando estaba en la otra. 238 nunca había que decir qué les había; no sé qué me se tenía, y qué le habían de mallar al salir, ni


In [40]:
generator("Sin pensarlo tuve que", max_length=50)

[{'generated_text': 'Sin pensarlo tuve que hacer una guerra y por eso yo me trago el puesto más bien de la espada. 352 a los lomos la cabeza me dio cuenta de ellas y se puso de blanco en mi baqueza me las panza'}]

In [45]:
generator("Puedo volar, dijo Manuel")

[{'generated_text': 'Puedo volar, dijo Manuel, y al fin lo hizo. 377 ya he hecho el favor a la vida noble, que me diecté a perseguir. 368 para los más penquistas no les pido ni más, que me hizo'}]

## Ejercicios adicionales

Juegen :)

In [36]:
from transformers import pipeline

generator = pipeline('text-generation', model="datificate/gpt2-small-spanish")
generator("Yo nunca habia escuchado hablar de Lituania hasta", max_length=50)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[{'generated_text': 'Yo nunca habia escuchado hablar de Lituania hasta el año 2096, cuando Lituania firmó un acuerdo secreto con los rusos, al que se le conoció como Pacto Ekaterina.\n\n Lituania estableció relaciones diplomáticas y la influencia rusa sobre el resto del'}]

In [34]:
generator("Puedo volar, dijo Manuel")

[{'generated_text': 'Puedo volar, dijo Manuel García-Rotelo.\n\nComo producto del éxito de la serie se le ofreció la posibilidad de crear una nueva película. Finalmente, se aprobó en 2001 una nueva temporada para las nuevas series. El actor Carlos Bartra'}]